<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

<img src = "https://github.com/Jangrae/img/blob/master/attrition.png?raw=true" width=800 align="left"/>

# 실습 내용

- 다양한 알고리즘으로 성능이 최적화된 모델을 만듭니다.
- 성능이 좋을 것으로 예상되는 모델을 선정해 평가를 진행합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/Attrition_simple2.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

**데이터 설명**

- **Attrition: 이직 여부 (1: 이직, 0: 잔류)**
- Age: 나이
- DistanceFromHome: 집-직장 거리 (단위: 마일)
- EmployeeNumber: 사번
- Gender: 성별 (Male, Female)
- JobSatisfaction: 직무 만족도(1: Low, 2: Medium, 3: High, 4: Very High)
- MaritalStatus: 결혼 상태 (Single, Married, Divorced)
- MonthlyIncome: 월급 (단위: 달러)
- OverTime: 야근 여부 (Yes, No)
- PercentSalaryHike: 전년 대비 급여 인상율(단위: %)
- TotalWorkingYears: 총 경력 연수

In [ ]:
# 데이터 살펴보기
data.head()

In [ ]:
# 기술통계 확인
data.describe()

In [ ]:
# 변수 관련 정보 확인
data.info()

In [ ]:
# 상관관계 
data.corr(numeric_only=True).style.background_gradient()

In [ ]:
# Target 비율
plt.figure(figsize=(8, 3))
sns.countplot(x=data['Attrition'])
plt.show()

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) 변수 제거**

In [ ]:
# 변수제거: EmployeeNumber
drop_cols = ['EmployeeNumber']
data.drop(columns=drop_cols, inplace=True)

# 확인
data.head()

**2) x, y 분리**

In [ ]:
# x, y 분리
target = 'Attrition'
x = data.drop(columns=target)
y = data.loc[:, target]

**2) 가변수화**

In [ ]:
# 가변수화
dumm_cols = ['Gender', 'JobSatisfaction', 'MaritalStatus', 'OverTime']
x = pd.get_dummies(x, columns=dumm_cols, drop_first=True, dtype=int)

# 확인
x.head()

**3) 학습용, 평가용 데이터 분리**

In [ ]:
# 학습용, 평가용 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

**4) 정규화**

In [ ]:
# 모듈 불러오기
from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)

# 4.모델링

- 하이퍼파라미터 최적화 과정을 통해 최선의 성능을 갖는 모델을 만들고 성능을 검증합니다.
- 각 모델을 joblib.dump() 함수를 사용해 저장합니다.

In [ ]:
# xgboost 설치
# !pip install xgboost

In [ ]:
# lightgbm 설치
# !pip install lightgbm

- 이후 사용할 함수를 모두 불러옵니다.

In [ ]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

**1) KNN**

- 최적으로 파라미터로 학습된 KNN 모델을 만들고, 최고의 파라미터와 성능을 확인합니다.
- 모델 이름은 model_dst로 지정합니다.

In [ ]:
# 파라미터 선언
param = {'n_neighbors': range(2, 11)}

# 선언하기
model_knn = GridSearchCV(KNeighborsClassifier(),
                         param,
                         cv=5)

# 학습하기
model_knn.fit(x_train_s, y_train)

# 결과확인
print('* 파라미터:', model_knn.best_params_)
print('* 예측성능:', model_knn.best_score_)

**2) Decision Tree**

- 최적으로 파라미터로 학습된 Decision Tree 모델을 만들고, 최고의 파라미터와 성능을 확인합니다.
- 모델 이름은 model_dst로 지정합니다.

In [ ]:
# 파라미터 선언
param = {'max_depth': range(1, 21)}

# 선언하기
model_dst = GridSearchCV(DecisionTreeClassifier(), param, cv=5)

# 학습하기
model_dst.fit(x_train, y_train)

# 결과확인
print('* 파라미터:', model_dst.best_params_)
print('* 예측성능:', model_dst.best_score_)

**3) Random Forest**

- 최적으로 파라미터로 학습된 Random Forest 모델을 만들고, 최고의 파라미터와 성능을 확인합니다.
- 모델 이름은 model_rdf로 지정합니다.

In [ ]:
# 파라미터 선언
param = {'max_depth': range(1, 21)}

# 선언하기
model_rdf = GridSearchCV(RandomForestClassifier(), param, cv=5)

# 학습하기
model_rdf.fit(x_train, y_train)

# 결과확인
print('* 파라미터:', model_rdf.best_params_)
print('* 예측성능:', model_rdf.best_score_)

**4) LightGBM**

- 최적으로 파라미터로 학습된 LightGBM 모델을 만들고, 최고의 파라미터와 성능을 확인합니다.
- 모델 이름은 model_lgb로 지정합니다.

In [ ]:
# 파라미터 선언
param = {'max_depth': range(1, 21)}

# 선언하기
model_rgb = GridSearchCV(LGBMClassifier(verbose=-1), param, cv=5)

# 학습하기
model_rgb.fit(x_train, y_train)

# 결과확인
print('* 파라미터:', model_rgb.best_params_)
print('* 예측성능:', model_rgb.best_score_)

# 5.성능평가

- 가장 좋은 성능을 보일 것으로 예상되는 모델로 평가를 진행합니다.

In [ ]:
# 예측하기
y_pred = model_rgb.predict(x_test)

# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))